In [1]:
import h2o

h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,1 day 8 hours 44 mins
H2O_cluster_timezone:,Asia/Kolkata
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.1
H2O_cluster_version_age:,9 days
H2O_cluster_name:,H2O_started_from_R_Aditya_Jain_hwc841
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,6.929 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [2]:
url = 'http://H2O-public-test-data.s3.amazonaws.com/smalldata/iris/iris_wheader.csv'
data = h2o.import_file(url)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [3]:
train, test = data.split_frame([0.8])

In [4]:
train.summary()

,sepal_len,sepal_wid,petal_len,petal_wid,class
type,real,real,real,real,enum
mins,4.3,2.0,1.0,0.1,
mean,5.926446280991737,3.021487603305785,3.9677685950413224,1.2826446280991732,
maxs,7.9,4.2,6.9,2.5,
sigma,0.8551577432498756,0.4137645489023617,1.73326353430136,0.7442756753997377,
zeros,0,0,0,0,
missing,0,0,0,0,0
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.6,3.1,1.5,0.2,Iris-setosa


In [6]:
train.nrow

121

In [7]:
test.nrow

29

#### Considerations

- AutoML takes 15% of train data for validation.
- AutoML can also accept a Leaderboard object as input

In [8]:
from h2o.automl import H2OAutoML

estimator = H2OAutoML(max_runtime_secs=20)
estimator.train(["sepal_len","sepal_wid","petal_len","petal_wid"],"class",train)

AutoML progress: |█
23:46:21.462: AutoML: XGBoost is not available; skipping it.

███████████████████████████████████████████████████████| 100%

23:46:28.620: Skipping training of model GBM_5_AutoML_20200412_234621 due to exception: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for GBM model: GBM_5_AutoML_20200412_234621.  Details: ERRR on field: _min_rows: The dataset size is too small to split for min_rows=100.0: must have at least 200.0 (weighted) rows, but have only 121.0.




In [9]:
estimator.leaderboard

model_id,mean_per_class_error,logloss,rmse,mse
GLM_1_AutoML_20200412_234621,0.0235065,0.0615812,0.133612,0.0178522
GBM_grid__1_AutoML_20200412_234621_model_31,0.038883,0.641294,0.474058,0.224731
DeepLearning_grid__1_AutoML_20200412_234621_model_1,0.038883,0.0955113,0.16825,0.0283079
GBM_grid__1_AutoML_20200412_234621_model_29,0.0461294,0.190216,0.218066,0.0475528
GBM_grid__1_AutoML_20200412_234621_model_3,0.0461294,0.1482,0.196145,0.038473
StackedEnsemble_BestOfFamily_AutoML_20200412_234621,0.0461294,0.242472,0.233677,0.0546049
GBM_grid__1_AutoML_20200412_234621_model_11,0.0470131,0.193465,0.215017,0.0462323
GBM_grid__1_AutoML_20200412_234621_model_23,0.0470131,0.210669,0.226649,0.0513696
GBM_grid__1_AutoML_20200412_234621_model_2,0.052492,0.178317,0.214619,0.0460615
DRF_1_AutoML_20200412_234621,0.0533758,0.386919,0.198925,0.0395713


In [10]:
pred = estimator.leader.predict(test)

glm prediction progress: |████████████████████████████████████████████████| 100%


In [11]:
pred

predict,Iris-setosa,Iris-versicolor,Iris-virginica
Iris-setosa,0.999604,0.000395699,8.9002e-19
Iris-setosa,0.999709,0.000290944,1.66642e-18
Iris-setosa,0.999602,0.000397747,4.81289e-21
Iris-setosa,0.999918,8.19096e-05,2.15297e-20
Iris-setosa,0.998473,0.00152747,5.02057e-17
Iris-setosa,0.999184,0.000815523,8.07876e-19
Iris-setosa,0.998728,0.00127159,9.32686e-19
Iris-setosa,0.999395,0.000605373,7.41572e-18
Iris-setosa,0.998312,0.00168762,2.13692e-17
Iris-setosa,0.992347,0.00765325,5.6677e-17


In [12]:
estimator.leader.model_performance(test)


ModelMetricsMultinomialGLM: glm
** Reported on test data. **

MSE: 0.01905516791381111
RMSE: 0.13804045752536143
